# Wess Zumino 

## N=1 in 1+1

The Hamiltonian with a central difference for the derivative is given by 

\begin{align}
    H=\sum_n&\left[ \frac{p_n^2}{2a} + \frac{a}{2}\left(\frac{\phi_{n+1}-\phi_{n-1}}{2a}\right)^2+\frac{a}{2}V(\phi_n)^2
            +aV(\phi_n)\frac{\phi_{n+1}-\phi_{n-1}}{2a} \right. \nonumber \\
            &\left.+(-1)^nV'(\phi_n)\left(\chi_n^{\dagger}\chi_n-\frac{1}{2}\right)
            +\frac{1}{2a}\left(\chi_n^{\dagger}\chi_{n+1}+\chi_{n+1}^{\dagger}\chi_n\right) \right],
\end{align}

In [1]:
import sympy as sp
from ham_to_matrix import *
from constants import *


In [2]:
bosonNI=sp.expand( pn**2/(2*aLat) + (aLat/2)*((qnP1-qnM1)/(2*aLat))**2 )
bosonI=sp.expand( (aLat/2)*V(qn)**2 + aLat*V(qn)*(qnP1-qnM1)/(2*aLat) )
fermionNI=sp.expand( (xdnP1*xn+xdn*xnP1)/(2*aLat) )
fermionI=sp.expand( sp.diff(V(qn),qn)*(xdn*xn-(1/2)) )

In [3]:
bosonNI+bosonI

a*V(q_{n})**2/2 + V(q_{n})*q_{n+1}/2 - V(q_{n})*q_{n-1}/2 + p_{n}**2/(2*a) - q_{n+1}*q_{n-1}/(8*a) + q_{n+1}**2/(8*a) - q_{n-1}*q_{n+1}/(8*a) + q_{n-1}**2/(8*a)

In [4]:
fermionNI + fermionI

-0.5*Derivative(V(q_{n}), q_{n}) + Derivative(V(q_{n}), q_{n})*\chi^{\dagger}_{n}*\chi_{n} + \chi^{\dagger}_{n+1}*\chi_{n}/(2*a) + \chi^{\dagger}_{n}*\chi_{n+1}/(2*a)

In [5]:
N=3
aVal=1

# depends on finite-difference method
qs=[SiteSymbol('q',str(i)) for i in range(-1,N+1,1)]
# maybe make qs a normal site list
# and make an extra boundaryQs list for q[-1], q[N]
# counting would be normal computer science way for rest of code.

ps=[SiteSymbol('p',str(i)) for i in range(-1,N+1,1)] # don't really need extras
aops=[SiteSymbol('a',str(i)) for i in range(-1,N+1,1)]
adags=[SiteSymbol('a^{\dagger}',str(i)) for i in range(-1,N+1,1)]
xs=[SiteSymbol('\chi',str(i)) for i in range(-1,N+1,1)]
xdags=[SiteSymbol('\chi^{\dagger}',str(i)) for i in range(-1,N+1,1)]

# note this is exactly hardcoded for this finite difference method.
bcType = 'periodic'
boundaryConditions = {}
if bcType == 'periodic':
    boundaryConditions = {qs[0]: qs[N], qs[N+1]: qs[1],
                          xs[0]: -xs[N], xs[N+1]: -xs[1], 
                          xdags[0]: -xdags[N],  xdags[N+1]: -xdags[1]
                         }
    
elif bcType == 'dirichlet':
    boundaryConditions = { qs[0]: 0, qs[N+1]: 0,
                          xs[0]: 0, xs[N+1]: 0,
                          xdags[0]: 0, xdags[N+1]:0
                         }

In [6]:
totHam=bosonNI+bosonI+fermionNI+fermionI
totHam

a*V(q_{n})**2/2 + V(q_{n})*q_{n+1}/2 - V(q_{n})*q_{n-1}/2 - 0.5*Derivative(V(q_{n}), q_{n}) + Derivative(V(q_{n}), q_{n})*\chi^{\dagger}_{n}*\chi_{n} + \chi^{\dagger}_{n+1}*\chi_{n}/(2*a) + \chi^{\dagger}_{n}*\chi_{n+1}/(2*a) + p_{n}**2/(2*a) - q_{n+1}*q_{n-1}/(8*a) + q_{n+1}**2/(8*a) - q_{n-1}*q_{n+1}/(8*a) + q_{n-1}**2/(8*a)

In [7]:
ham=0

lam=1

for i in range(1,N+1):
    ham+=(bosonNI+bosonI+fermionNI).subs({
        pn: ps[i],
        qn: qs[i], qnP1: qs[i+1], qnM1: qs[i-1],
        xn: xs[i], xnP1: xs[i+1],
        xdn: xdags[i], xdnP1: xdags[i+1]
    }).subs(boundaryConditions)
    
# ham.subs(boundaryConditions).doit() # this doesn't work?


def potential(n):
    # m*q with m=1
    return lam*qs[n]

potentialSubs={}
for n in range(1,N+1):
    potentialSubs[V(qs[n])]=potential(n)

ham=sp.simplify(ham.subs(potentialSubs).subs(aLat,aVal))

for i in range(1,N+1):
    if i%2==0:
        ham+=sp.simplify(fermionI.subs({qn: qs[i], xn: xs[i], xdn: xdags[i]}).subs(potentialSubs))
    else:
        ham-=sp.simplify(fermionI.subs({qn: qs[i], xn: xs[i], xdn: xdags[i]}).subs(potentialSubs))
ham=sp.simplify(ham.subs(aLat,aVal))
ham

(4.0 - 8*\chi^{\dagger}_{0}*\chi_{0} + 4*\chi^{\dagger}_{0}*\chi_{1} - 4*\chi^{\dagger}_{0}*\chi_{2} + 4*\chi^{\dagger}_{1}*\chi_{0} + 8*\chi^{\dagger}_{1}*\chi_{1} + 4*\chi^{\dagger}_{1}*\chi_{2} - 4*\chi^{\dagger}_{2}*\chi_{0} + 4*\chi^{\dagger}_{2}*\chi_{1} - 8*\chi^{\dagger}_{2}*\chi_{2} + 4*p_{0}**2 + 4*p_{1}**2 + 4*p_{2}**2 + 3*q_{0}*q_{1} - 5*q_{0}*q_{2} + 6*q_{0}**2 - 5*q_{1}*q_{0} + 3*q_{1}*q_{2} + 6*q_{1}**2 + 3*q_{2}*q_{0} - 5*q_{2}*q_{1} + 6*q_{2}**2)/8

In [8]:
#ham=0
#totHam=bosonNI+bosonI+fermionNI+fermionI
#for i in range(0,N):
#    ham+=totHam.subs(n,i).subs(boundaryConditions)
# ham.subs(boundaryConditions).doit() # this doesn't work?

#def potential(n):
#    return q[n]

#for n in range(0,N):
#    ham=ham.subs(V(q[n]),potential(n))
#ham=ham.subs(aLat,aVal).simplify().expand()
#ham

In [9]:
m=1

HOdofSubs = {}
#offset because of BC
for i in range(1,N+1):
    HOdofSubs[qs[i]] = 0.5*sp.sqrt(2/m)*(aops[i] + adags[i])
    HOdofSubs[ps[i]] = complex(0,1)*sp.sqrt(2*m)*(adags[i] - aops[i])/2 

hoHam=sp.expand(ham.subs(HOdofSubs))
hoHam=sp.nsimplify(hoHam,tolerance=1e-8)
hoHam

1/2 - \chi^{\dagger}_{0}*\chi_{0} + \chi^{\dagger}_{0}*\chi_{1}/2 - \chi^{\dagger}_{0}*\chi_{2}/2 + \chi^{\dagger}_{1}*\chi_{0}/2 + \chi^{\dagger}_{1}*\chi_{1} + \chi^{\dagger}_{1}*\chi_{2}/2 - \chi^{\dagger}_{2}*\chi_{0}/2 + \chi^{\dagger}_{2}*\chi_{1}/2 - \chi^{\dagger}_{2}*\chi_{2} + 3*a^{\dagger}_{0}*a^{\dagger}_{1}/16 - 5*a^{\dagger}_{0}*a^{\dagger}_{2}/16 + 5*a^{\dagger}_{0}*a_{0}/8 + 3*a^{\dagger}_{0}*a_{1}/16 - 5*a^{\dagger}_{0}*a_{2}/16 + a^{\dagger}_{0}**2/8 - 5*a^{\dagger}_{1}*a^{\dagger}_{0}/16 + 3*a^{\dagger}_{1}*a^{\dagger}_{2}/16 - 5*a^{\dagger}_{1}*a_{0}/16 + 5*a^{\dagger}_{1}*a_{1}/8 + 3*a^{\dagger}_{1}*a_{2}/16 + a^{\dagger}_{1}**2/8 + 3*a^{\dagger}_{2}*a^{\dagger}_{0}/16 - 5*a^{\dagger}_{2}*a^{\dagger}_{1}/16 + 3*a^{\dagger}_{2}*a_{0}/16 - 5*a^{\dagger}_{2}*a_{1}/16 + 5*a^{\dagger}_{2}*a_{2}/8 + a^{\dagger}_{2}**2/8 + 5*a_{0}*a^{\dagger}_{0}/8 + 3*a_{0}*a^{\dagger}_{1}/16 - 5*a_{0}*a^{\dagger}_{2}/16 + 3*a_{0}*a_{1}/16 - 5*a_{0}*a_{2}/16 + a_{0}**2/8 - 5*a_{1}*a^{\da

In [10]:
from ham_to_sparse_matrix import *

t=Timer('sparse data storage')
t.start()
cutoff=4
term=hoHam.args[4]
print(term)
hamMat=convert_term_to_matrix(term,cutoff,N,aops,adags,xs,xdags)
t.stop()
hamMat

a_{0}**2/8
sparse data storage timer took: 0.0210 seconds


<512x512 sparse matrix of type '<class 'numpy.object_'>'
	with 256 stored elements in Compressed Sparse Row format>

In [11]:
from ham_to_sparse_matrix import *

t=Timer('sparse data storage')
t.start()
cutoff=4

print(term)
hamMat=convert_to_matrix(hoHam,cutoff,N,aops,adags,xs,xdags)
t.stop()
hamMat

a_{0}**2/8


/home/chris/anaconda3/envs/QuantumComputing/lib/python3.9/site-packages/scipy/sparse/_sputils.py:114: UserWarning: object dtype is not supported by sparse matrices
  warnings.warn("object dtype is not supported by sparse matrices")


sparse data storage timer took: 1.1378 seconds


<512x512 sparse matrix of type '<class 'numpy.complex64'>'
	with 6272 stored elements in Compressed Sparse Row format>

In [13]:
hamMat.toarray()

array([[2.375+0.j, 0.   +0.j, 0.   +0.j, ..., 0.   +0.j, 0.   +0.j,
        0.   +0.j],
       [0.   +0.j, 1.375+0.j, 0.   +0.j, ..., 0.   +0.j, 0.   +0.j,
        0.   +0.j],
       [0.   +0.j, 0.   +0.j, 3.625+0.j, ..., 0.   +0.j, 0.   +0.j,
        0.   +0.j],
       ...,
       [0.   +0.j, 0.   +0.j, 0.   +0.j, ..., 6.375+0.j, 0.   +0.j,
        0.   +0.j],
       [0.   +0.j, 0.   +0.j, 0.   +0.j, ..., 0.   +0.j, 6.125+0.j,
        0.   +0.j],
       [0.   +0.j, 0.   +0.j, 0.   +0.j, ..., 0.   +0.j, 0.   +0.j,
        5.125+0.j]], dtype=complex64)

In [14]:
ens=np.sort(np.linalg.eig(hamMat.toarray().astype(np.complex64))[0])
ens=ens.round(8)

print(np.isreal(ens).all())

ens[0:10]

True


array([0.04155126+0.j, 0.636187  +0.j, 0.85696435+0.j, 1.070188  +0.j,
       1.070188  +0.j, 1.3223276 +0.j, 1.4516002 +0.j, 1.4790901 +0.j,
       1.59579   +0.j, 1.59579   +0.j], dtype=complex64)

In [13]:
from ham_to_matrix import *

t=Timer('np data storage')
t.start()
cutoff=4
term=hoHam.args[4]
print(term)
hamMat=convert_term_to_matrix(term,cutoff,N,aops,adags,xs,xdags)
t.stop()
hamMat


a_{0}**2/8
np data storage timer took: 13.5366 seconds


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=object)

In [11]:
a2*8

NameError: name 'a2' is not defined

In [12]:
aMat(4)

array([[0.        , 1.        , 0.        , 0.        ],
       [0.        , 0.        , 1.41421356, 0.        ],
       [0.        , 0.        , 0.        , 1.73205081],
       [0.        , 0.        , 0.        , 0.        ]])

In [17]:
a2=np.linalg.matrix_power(aMat(4),2)

In [19]:
sa2=scipy.sparse.coo_matrix(a2)

In [20]:
full=scipy.sparse.coo_matrix([1])

In [21]:
tmp=scipy.sparse.kron(full,sa2)

In [22]:
sid=scipy.sparse.coo_matrix(np.eye(2))

In [25]:
tmp2=scipy.sparse.kron(tmp,sid)

In [26]:
tmp2

<8x8 sparse matrix of type '<class 'numpy.float64'>'
	with 8 stored elements (blocksize = 2x2) in Block Sparse Row format>

In [28]:
print(tmp)

  (0, 2)	1.4142135623730951
  (1, 3)	2.4494897427831783


In [47]:
tst=np.kron(a2,np.eye(4))

In [50]:
n1=0
n2=0
for row in tst:
    for elem in row:
        if abs(a2[0,2]-elem)<1e-6:
            n1+=1
        if abs(a2[1,3]-elem)<1e-6:
            n2+=1
print(n1)
print(n2)

4
4


In [15]:
a2[0,2]

0.1767766952966369

In [16]:
a2[1,3]

0.3061862178478973

In [17]:
hamMat.shape

(512, 512)

In [51]:
n1=0
n2=0
for row in hamMat:
    for elem in row:
        if abs(a2[0,2]-elem)<1e-6:
            n1+=1
        if abs(a2[1,3]-elem)<1e-6:
            n2+=1
print(n1)
print(n2)

128
128


In [11]:
import scipy

In [12]:
amat=scipy.sparse.coo_matrix(aMat(4))
adagmat=scipy.sparse.coo_matrix(aDagMat(4))
iden=scipy.sparse.coo_matrix(np.eye(4))

namat=aMat(4)
nadagmat=aDagMat(4)
niden=np.eye(4)

In [13]:
def skron(a,b):
    return scipy.sparse.kron(a,b)

def nkron(a,b):
    return np.kron(a,b)

In [14]:
t=Timer('skron')
t.start()
for i in range(0,100):
    sres=skron(skron(skron(skron(skron(amat,adagmat),iden),amat),iden),adagmat)
t.stop()

skron timer took: 0.0473 seconds


In [15]:
t=Timer('nkron')
t.start()
for i in range(0,10):
    nres=nkron(nkron(nkron(nkron(nkron(namat,nadagmat),niden),namat),niden),nadagmat)
t.stop()

nkron timer took: 1.7117 seconds


In [16]:
(sres==nres).all

<bound method matrix.all of matrix([[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]])>

In [17]:
sres.shape

(4096, 4096)

In [18]:
nres.shape

(4096, 4096)